In [1]:
import random
from typing import Callable, Optional, TypedDict
import numpy as np
from obp.dataset import SyntheticBanditDataset
from obp.ope import ReplayMethod, InverseProbabilityWeighting
import polars as pl

/Users/masato.morita/Library/Caches/pypoetry/virtualenvs/recommender-experiments-_FVBVT8O-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class BanditFeedbackDict(TypedDict):
    n_rounds: int  # ラウンド数
    n_actions: int  # アクション数
    context: np.ndarray  # 文脈 (shape: (n_rounds, dim_context))
    action_context: (
        np.ndarray
    )  # アクション特徴量 (shape: (n_actions, dim_action_features))
    action: np.ndarray  # 実際に選択されたアクション (shape: (n_rounds,))
    position: Optional[np.ndarray]  # ポジション (shape: (n_rounds,) or None)
    reward: np.ndarray  # 報酬 (shape: (n_rounds,))
    expected_reward: np.ndarray  # 期待報酬 (shape: (n_rounds, n_actions))
    pi_b: np.ndarray  # データ収集方策 P(a|x) (shape: (n_rounds, n_actions))
    pscore: np.ndarray  # 傾向スコア (shape: (n_rounds,))

In [6]:
# 真の期待報酬関数 E_{p(r|x,a)}[r] を定義する
def expected_reward_function(
    context: np.ndarray,
    action_context: np.ndarray,
    random_state: int = None,
) -> np.ndarray:
    """(アクションa, 文脈x)の各組み合わせに対する期待報酬 E_{p(r|x,a)}[r] を定義する関数
    今回の場合は、推薦候補4つの記事を送った場合の報酬rの期待値を、文脈xに依存しない固定値として設定する
    ニュース0: 0.2, ニュース1: 0.15, ニュース2: 0.1, ニュース3: 0.05
    返り値のshape: (n_rounds, n_actions, len_list)
    """
    n_rounds = context.shape[0]
    n_actions = action_context.shape[0]

    # 固定の期待報酬を設定 (n_actions=4として固定値を設定)
    fixed_rewards = np.array([0.4, 0.3, 0.2, 0.2])

    # 文脈の数だけ期待報酬を繰り返して返す
    return np.tile(fixed_rewards, (n_rounds, 1))


# 試しに期待報酬関数を実行してみる
n_rounds = 3
n_actions = 4
context = np.array([[1], [2], [3]])
action_context = np.array([[1], [2], [3], [4]])
print(expected_reward_function(context, action_context))

[[0.4 0.3 0.2 0.2]
 [0.4 0.3 0.2 0.2]
 [0.4 0.3 0.2 0.2]]


In [15]:
# 5種類の推薦方策を定義していく


from typing import Literal


def pi_1(
    context: np.ndarray,
    action_context: np.ndarray,
    random_state: int = None,
) -> np.ndarray:
    """コンテキストを考慮せず、全てのユーザに対してニュース $a_0$ を確率1で推薦する決定的方策"""
    n_rounds = context.shape[0]
    n_actions = action_context.shape[0]

    p_scores = np.array([1.0, 0.0, 0.0, 0.0])
    action_dist = np.tile(p_scores, (n_rounds, 1))

    assert action_dist.shape == (n_rounds, n_actions)
    return action_dist


def pi_2(
    context: np.ndarray,
    action_context: np.ndarray,
    random_state: int = None,
) -> np.ndarray:
    """コンテキストを考慮せず、全てのユーザに対してニュース $a_1$ を確率1で推薦する決定的方策"""
    n_rounds = context.shape[0]
    n_actions = action_context.shape[0]

    p_scores = np.array([0.0, 1.0, 0.0, 0.0])
    action_dist = np.tile(p_scores, (n_rounds, 1))

    assert action_dist.shape == (n_rounds, n_actions)
    return action_dist


def pi_3(
    context: np.ndarray,
    action_context: np.ndarray,
    random_state: int = None,
) -> np.ndarray:
    """コンテキストを考慮せず、全てのユーザに対してニュース $a_0$ を確率0.4、
    ニュース $a_1$ を確率0.3、ニュース $a_2$ を確率0.2、ニュース $a_3$ を確率0.1で
    推薦する確率的方策"""
    n_rounds = context.shape[0]
    n_actions = action_context.shape[0]

    p_scores = np.array([0.4, 0.3, 0.2, 0.1])
    action_dist = np.tile(p_scores, (n_rounds, 1))

    assert action_dist.shape == (n_rounds, n_actions)
    return action_dist


def pi_4(
    context: np.ndarray,
    action_context: np.ndarray,
    random_state: int = None,
) -> np.ndarray:
    """ユーザとニュースのコンテキストを考慮し、
    コンテキストベクトル $x$ とアイテムコンテキストベクトル $e$ の内積が最も大きいニュースを
    確率1で推薦する決定的方策"""
    n_rounds = context.shape[0]
    n_actions = action_context.shape[0]

    # 内積を計算
    scores = context @ action_context.T  # shape: (n_rounds, n_actions)

    # 各ラウンドで最もスコアが高いアクションのindexを取得
    selected_actions = np.argmax(scores, axis=1)  # shape: (n_rounds,)

    # 決定的方策: 確率1で最もスコアが高いアクションを選択
    action_dist = np.zeros((n_rounds, n_actions))
    action_dist[np.arange(n_rounds), selected_actions] = 1.0

    return action_dist


def pi_5(
    context: np.ndarray,
    action_context: np.ndarray,
    random_state: int = None,
) -> np.ndarray:
    """ユーザとニュースのコンテキストを考慮し、
    コンテキストベクトル $x$ とアイテムコンテキストベクトル $e$ の内積が最も大きいニュースを
    確率0.7で推薦し、その他のニュースを均等に確率0.1で推薦する確率的方策。
    """
    n_rounds = context.shape[0]
    n_actions = action_context.shape[0]

    # 内積を計算
    scores = context @ action_context.T  # shape: (n_rounds, n_actions)

    # 各ラウンドで最もスコアが高いアクションのindexを取得
    selected_actions = np.argmax(scores, axis=1)  # shape: (n_rounds,)

    # 確率的方策: 確率0.4で全てのアクションを一様ランダムに選択し、確率0.6で最もスコアが高いアクションを決定的に選択
    action_dist = np.full((n_rounds, n_actions), 0.1)
    action_dist[np.arange(n_rounds), selected_actions] = 0.7
    return action_dist


print(
    pi_5(
        context=np.array([[1], [2], [3]]),
        action_context=np.array([[1], [2], [3], [4]]),
    )
)

[[0.1 0.1 0.1 0.7]
 [0.1 0.1 0.1 0.7]
 [0.1 0.1 0.1 0.7]]


In [16]:
def policy_wrapper(
    base_policy: Callable[[np.ndarray, np.ndarray, int], np.ndarray],
    mode: Literal["logging_poicy", "target_policy"],
) -> Callable[[np.ndarray, np.ndarray, int], np.ndarray]:
    """
    方策のラッパー関数。データ収集時と評価時で出力の形状を切り替える。
    (logging_policyの場合はshape=(n_rounds, n_actions)、target_policyの場合はshape=(n_rounds,n_actions, 1))
    Parameters
    - base_policy: ベースとなる方策関数
    - mode: 方策のモード。"logging_policy"はデータ収集方策、"target_policy"は評価方策

    Returns
    - wrapされた方策関数
    """

    def wrapped_policy(
        context: np.ndarray,
        action_context: np.ndarray,
        random_state: int = None,
    ) -> np.ndarray:
        action_dist = base_policy(context, action_context, random_state)
        if mode == "logging_policy":
            return action_dist
        elif mode == "target_policy":
            return action_dist[:, :, np.newaxis]

    return wrapped_policy


target_policy = policy_wrapper(pi_5, mode="target_policy")
print(target_policy(context, action_context))
print(target_policy(context, action_context).shape)

[[[0.1]
  [0.1]
  [0.1]
  [0.7]]

 [[0.1]
  [0.1]
  [0.1]
  [0.7]]

 [[0.1]
  [0.1]
  [0.1]
  [0.7]]]
(3, 4, 1)
